In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

# Data Fetching

In [3]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16])
    out= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19])
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# Min Max Scaler

In [4]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(A1)
Y1=scaler_obj.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

X1=X1[:,np.newaxis,:]
Y1=Y1[:,np.newaxis,:]

In [5]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Model

In [6]:
model1 = Sequential()
model1.add(keras.Input(shape=(1,5)))
model1.add(tf.keras.layers.LSTM(7,activation="tanh",use_bias=True,kernel_initializer="glorot_uniform",bias_initializer="zeros"))
model1.add(Dense(7))
model1.add(keras.layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True,
                                beta_initializer="zeros",gamma_initializer="ones",
                                moving_mean_initializer="zeros",moving_variance_initializer="ones",trainable=True))
model1.add(keras.layers.ReLU())
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy',metrics=['accuracy','mse','mae',rmse])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 7)                 364       
_________________________________________________________________
dense (Dense)                (None, 7)                 56        
_________________________________________________________________
batch_normalization (BatchNo (None, 7)                 28        
_________________________________________________________________
re_lu (ReLU)                 (None, 7)                 0         
Total params: 448
Trainable params: 434
Non-trainable params: 14
_________________________________________________________________


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=42)

In [7]:
model_fit8 = model1.fit(x_train,y_train,batch_size=2048,epochs=300, validation_split=0.1)

Epoch 1/300
571/571 [==============================] - 434s 757ms/step - loss: 0.2997 - accuracy: 0.1779 - mse: 0.0372 - mae: 0.1334 - rmse: 0.1780 - val_loss: 0.1868 - val_accuracy: 0.1774 - val_mse: 0.0132 - val_mae: 0.0736 - val_rmse: 0.1027
Epoch 2/300
571/571 [==============================] - 430s 754ms/step - loss: 0.2572 - accuracy: 0.1616 - mse: 0.0329 - mae: 0.1199 - rmse: 0.1614 - val_loss: 0.2287 - val_accuracy: 0.1387 - val_mse: 0.0295 - val_mae: 0.1116 - val_rmse: 0.1534
Epoch 3/300
571/571 [==============================] - 432s 756ms/step - loss: 0.2147 - accuracy: 0.0318 - mse: 0.0295 - mae: 0.1114 - rmse: 0.1541 - val_loss: 0.2002 - val_accuracy: 0.0045 - val_mse: 0.0260 - val_mae: 0.1025 - val_rmse: 0.1448
Epoch 4/300
571/571 [==============================] - 433s 758ms/step - loss: 0.1948 - accuracy: 0.0283 - mse: 0.0265 - mae: 0.1037 - rmse: 0.1462 - val_loss: 0.1848 - val_accuracy: 0.0521 - val_mse: 0.0236 - val_mae: 0.0958 - val_rmse: 0.1379
Epoch 5/300
571/571 

571/571 [==============================] - 439s 769ms/step - loss: 0.0714 - accuracy: 0.9438 - mse: 1.6052e-04 - mae: 0.0053 - rmse: 0.0085 - val_loss: 0.0714 - val_accuracy: 0.9439 - val_mse: 1.5920e-04 - val_mae: 0.0053 - val_rmse: 0.0085
Epoch 67/300
571/571 [==============================] - 438s 768ms/step - loss: 0.0714 - accuracy: 0.9438 - mse: 1.5992e-04 - mae: 0.0053 - rmse: 0.0084 - val_loss: 0.0714 - val_accuracy: 0.9439 - val_mse: 1.5985e-04 - val_mae: 0.0054 - val_rmse: 0.0086
Epoch 68/300
571/571 [==============================] - 438s 768ms/step - loss: 0.0714 - accuracy: 0.9438 - mse: 1.5924e-04 - mae: 0.0052 - rmse: 0.0084 - val_loss: 0.0714 - val_accuracy: 0.9439 - val_mse: 1.5735e-04 - val_mae: 0.0052 - val_rmse: 0.0083
Epoch 69/300
571/571 [==============================] - 437s 765ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5870e-04 - mae: 0.0052 - rmse: 0.0083 - val_loss: 0.0714 - val_accuracy: 0.9439 - val_mse: 1.5938e-04 - val_mae: 0.0055 - val_rmse: 0.00

571/571 [==============================] - 438s 767ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5370e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5305e-04 - val_mae: 0.0045 - val_rmse: 0.0075
Epoch 131/300
571/571 [==============================] - 436s 764ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5361e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5281e-04 - val_mae: 0.0047 - val_rmse: 0.0080
Epoch 132/300
571/571 [==============================] - 437s 766ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5357e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5248e-04 - val_mae: 0.0047 - val_rmse: 0.0079
Epoch 133/300
571/571 [==============================] - 437s 766ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5354e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5332e-04 - val_mae: 0.0044 - val_rmse: 0

571/571 [==============================] - 256s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5318e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5225e-04 - val_mae: 0.0046 - val_rmse: 0.0078
Epoch 195/300
571/571 [==============================] - 256s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5326e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5227e-04 - val_mae: 0.0046 - val_rmse: 0.0078
Epoch 196/300
571/571 [==============================] - 257s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5310e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5304e-04 - val_mae: 0.0047 - val_rmse: 0.0081
Epoch 197/300
571/571 [==============================] - 256s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5316e-04 - mae: 0.0046 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5219e-04 - val_mae: 0.0045 - val_rmse: 0

571/571 [==============================] - 256s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5293e-04 - mae: 0.0045 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5353e-04 - val_mae: 0.0048 - val_rmse: 0.0082
Epoch 259/300
571/571 [==============================] - 256s 448ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5296e-04 - mae: 0.0045 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5369e-04 - val_mae: 0.0045 - val_rmse: 0.0078
Epoch 260/300
571/571 [==============================] - 256s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5296e-04 - mae: 0.0045 - rmse: 0.0078 - val_loss: 0.0714 - val_accuracy: 0.9439 - val_mse: 1.5377e-04 - val_mae: 0.0043 - val_rmse: 0.0075
Epoch 261/300
571/571 [==============================] - 256s 449ms/step - loss: 0.0713 - accuracy: 0.9438 - mse: 1.5295e-04 - mae: 0.0045 - rmse: 0.0078 - val_loss: 0.0713 - val_accuracy: 0.9439 - val_mse: 1.5272e-04 - val_mae: 0.0044 - val_rmse: 0

# Saving Model as File

In [8]:
model_json = model1.to_json()
with open("lstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model1.save_weights("lstm.h5")
print("Saved model to disk")

Saved model to disk


In [8]:
from keras.models import model_from_json
json_file = open('lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("lstm.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse',metrics=['accuracy','mse','mae',rmse,coeff_determination])
print(loaded_model.evaluate(x_train, y_train, verbose=0))

Loaded model from disk
[0.00015189875557553023, 0.94383305311203, 0.00015189917758107185, 0.004441055003553629, 0.007611081004142761, -6.193653106689453]


# Error Analysis

In [9]:
# _, train_acc = model1.evaluate(x_train, y_train, verbose=0)
# _, test_acc = model1.evaluate(x_test, y_test, verbose=0)
# print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# summarize history for loss
model_fit8=loaded_model
plt.plot(model_fit8.history['loss'])
plt.plot(model_fit8.history['val_loss'])
plt.title('Model Loss',fontweight ='bold',fontsize = 15)
plt.ylabel('Loss',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(model_fit8.history['accuracy'])
plt.plot(model_fit8.history['val_accuracy'])
plt.title('Model accuracy',fontweight ='bold',fontsize = 15)
plt.ylabel('Accuracy',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

TypeError: 'History' object is not subscriptable